# Local Ocean Conservation Sea Turtle Face Detection Zindi Competition
## Taratra D. RAHARISON
### Notebook inspired by the Detectron2 tutorial

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import itertools
import logging
from pathlib import Path
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting
from sklearn.model_selection import train_test_split
import time
import datetime
import json
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/detectron2-sea-turtle-data-loader/__results__.html
/kaggle/input/detectron2-sea-turtle-data-loader/SampleSubmission.csv
/kaggle/input/detectron2-sea-turtle-data-loader/IMAGES_512.zip
/kaggle/input/detectron2-sea-turtle-data-loader/__notebook__.ipynb
/kaggle/input/detectron2-sea-turtle-data-loader/__output__.json
/kaggle/input/detectron2-sea-turtle-data-loader/Train.csv
/kaggle/input/detectron2-sea-turtle-data-loader/IMAGES_1024.zip
/kaggle/input/detectron2-sea-turtle-data-loader/custom.css


## Installing torch 1.7.1, torchvision 0.8.2, torchaudio 0.7.2

In [2]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1156.8 MB 8.1 kB/s             
     |████████████████████████████████| 12.9 MB 38.6 MB/s            
     |████████████████████████████████| 7.6 MB 1.8 MB/s            
  Attempting uninstall: torch
    Found existing installation: torch 1.9.1
    Uninstalling torch-1.9.1:
      Successfully uninstalled torch-1.9.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.1
    Uninstalling torchvision-0.10.1:
      Successfully uninstalled torchvision-0.10.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.9.1
    Uninstalling torchaudio-0.9.1:
      Successfully uninstalled torchaudio-0.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairscale 0.4.5 requires torch>=1.8.0, but you have tor

## Installing pyaml 5.1

In [3]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 4.2 MB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=a79db0d3ff44d0f44f86fd485f8bf9d0dcd044d3e91fc88e1a27b29d4c3861b7
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
libcst 0.3.23 requires pyyaml>=5.2, but you have pyyaml 5.1 which is incompatible.
kubernetes 21.7.0 requires pyyaml>=5.4.1, but you have pyyaml 5.1 which is incompatible.
dask 2022.2.0 requires pyyaml>=5.3.1,

## Installing Detectron2

In [4]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html


Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html
     |████████████████████████████████| 6.4 MB 4.2 MB/s            
     |████████████████████████████████| 79 kB 3.3 MB/s             
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     |████████████████████████████████| 130 kB 25.6 MB/s            
     |████████████████████████████████| 154 kB 55.4 MB/s            
     |████████████████████████████████| 50 kB 6.1 MB/s             
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 117 kB 51.8 MB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61433 sha256=8eb319b747c537e13fa0bc4567105abcc71840ab1c329fac31d582877f22c791
  Stored in directory: /root/.cache/pip/wheels/af/cd/23/3fb62ec8606c

## Loading Detectron2

In [5]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import cv2, random
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer,PeriodicCheckpointer

## Unzipping data

In [6]:
!unzip -q '/kaggle/input/detectron2-sea-turtle-data-loader/IMAGES_512.zip'

## Loading data

In [7]:
data = pd.read_csv('../input/detectron2-sea-turtle-data-loader/Train.csv')
# Splitting between train, val and test
train, testTp = train_test_split(data, test_size=0.2, random_state=42)

val, test_train = train_test_split(testTp, test_size=0.5, random_state=42)

## A glimpse inside train

In [8]:
train.head()

,Image_ID,x,y,w,h
170,CC029777,0.400825,0.599537,0.240668,0.282118
270,4661E536,0.473524,0.275174,0.175998,0.326389
890,81B7CD6B,0.462500,0.241667,0.520750,0.464333
209,9EC9DEC9,0.339844,0.478009,0.335938,0.357928
721,9C527060,0.483507,0.431713,0.289714,0.344039


## Getting the bounding boxes

In [9]:

def get_turtle_box(img_dir, dataset):
    """
    Converting from x,y,width,height boxes to x1,y1,X2,y2 boxes
    """
    dataset_dicts = []
    images = list(Path(img_dir).glob('*.JPG'))
    for i in range(len(images)):
        record = {}
        filename = str(images[i])
        #print(filename)
        image_name = filename[len(img_dir)-1:]
        #print(image_name)
        img = os.path.join(img_dir, image_name)
        #print(img)
        height, width = cv2.imread(img).shape[:2]
        
        #print(height,width)
        record["file_name"] = filename
        record["image_id"] = image_name[:-4]
        record["height"] = height
        record["width"] = width
        
        objs = []
        if any(dataset['Image_ID'] == record['image_id']):
            x1 = dataset[dataset['Image_ID'] == record['image_id']]['x'].values[0] * width
            y1 = dataset[dataset['Image_ID'] == record['image_id']]['y'].values[0] * height
            x2 =  x1 + dataset[dataset['Image_ID'] == record['image_id']]['w'].values[0] * width
            y2 = y1 + dataset[dataset['Image_ID'] == record['image_id']]['h'].values[0]* height
            
            obj = {
                "bbox": [x1, y1, x2, y2],
                #"segmentation": [poly],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": 0,
            }
            objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
    return dataset_dicts

## Registring datasets

In [10]:
image_folder = "./IMAGES_512"
d = 'train'
DatasetCatalog.register("turtle_" + d, lambda d=d: get_turtle_box(image_folder,train))
MetadataCatalog.get("turtle_" + d).set(thing_classes=["turtle"])
d = 'val'
DatasetCatalog.register("turtle_" + d, lambda d=d: get_turtle_box(image_folder,val))
MetadataCatalog.get("turtle_" + d).set(thing_classes=["turtle"])

namespace(name='turtle_val', thing_classes=['turtle'])

In [11]:
dataset_dicts = get_turtle_box(image_folder,train)
val_set_dicts = get_turtle_box(image_folder,val)

### Training and validation inspired by https://eidos-ai.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e

## Early stopping mechanism

HookBase is a class in Detectron2 that makes it possible to customize the training process. By subclassing Hookbase, we can extend the training process without changing the core training code of Detectron2.

The key methods defined in HookBase include:
* before_step(): This method is called before each training step.
* after_step(): This method is called after each training step.
* before_train(): This method is called before the training starts.
* after_train(): This method is called after the training ends.

Here, we want after each step to evaluate the model, see if the performance improved. If it is the case, save the new best model. If not, first check if we did not exceeded the patience parameter, if it is not the case, continue the training, if not, stop it.

We will implement those process inside after_step

In [12]:

class EarlyStoppingAndLossEvalHook(HookBase):
    def __init__(self, patience, validation_evaluator, validation_loss_key, eval_period, model, data_loader, output_dir):
        super().__init__()
        self.patience = patience
        self.validation_evaluator = validation_evaluator
        self.validation_loss_key = validation_loss_key
        self.best_loss = float('inf')
        self.counter = 0
        self.eval_period = eval_period
        self.model = model
        self.data_loader = data_loader
        self.output_dir = output_dir

    def after_step(self):
        """
        Defines the behavior after each step of the gradient descent
        Implements the early stopping mechanism
        """
        if (self.trainer.iter + 1) % self.trainer.cfg.SOLVER.CHECKPOINT_PERIOD != 0:
            return

        # Perform loss evaluation
        losses = self._do_loss_eval()

        # Get the segmentation loss from the evaluation results
        segmentation_loss = np.mean(losses)

        if segmentation_loss < self.best_loss:
            self.best_loss = segmentation_loss
            self.counter = 0
            # Save the best model
            self._save_best_model()
        else:
            self.counter += 1

        if self.counter >= self.patience:
            print(f"Early stopping triggered after {self.counter} steps without improvement.")
            self.trainer.should_stop = True
            # Load the best model
            self._load_best_model()

    def _do_loss_eval(self):
        total = len(self.data_loader)
        num_warmup = min(5, total - 1)

        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self.data_loader):
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        self.trainer.storage.put_scalar('validation_loss', np.mean(losses))
        comm.synchronize()

        return losses

    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self.model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced

    def _save_best_model(self):
        save_path = os.path.join(self.output_dir, "best_model.pth")
        torch.save(self.model.state_dict(), save_path)
        print(f"Best model saved at {save_path}")

    def _load_best_model(self):
        load_path = os.path.join(self.output_dir, "best_model.pth")
        if os.path.exists(load_path):
            self.model.load_state_dict(torch.load(load_path))
            print(f"Best model loaded from {load_path}")
        else:
            print("Best model checkpoint not found, training will continue with the current model")


## DefaultTrainer

DefaultTrainer is a class in Detectron2 that provides a default implementation of the training loop. It encapsulates the core logic for training object detection and instance segmentation models. We usually extend this class when we use custom hooks.

In [13]:
class MyTrainer(DefaultTrainer):
    def __init__(self, cfg):
        super().__init__(cfg)
        #self.output_dir = output_dir
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

    def build_hooks(self):
        hooks = super().build_hooks()
        early_stopping_hook = EarlyStoppingAndLossEvalHook(
            patience=5,
            validation_evaluator=self.build_evaluator(self.cfg, self.cfg.DATASETS.TEST[0]),
            validation_loss_key="validation_loss",
            eval_period=cfg.TEST.EVAL_PERIOD,
            model=self.model,
            data_loader=build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg, True)
            ),
            output_dir=self.cfg.OUTPUT_DIR
        )

        # Insert the new hooks
        hooks.insert(-1, early_stopping_hook)

        return hooks

## Getting configuration file and training the model

In [14]:

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("turtle_train",)
cfg.DATASETS.TEST = ("turtle_val",)
cfg.TEST.EVAL_PERIOD = 50
cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
# Let training initialize from model zoo
# Early stopping and checkpoint
cfg.SOLVER.CHECKPOINT_PERIOD = 50
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.REFERENCE_WORLD_SIZE = 2
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 350    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (turtle). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg)
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[07/09 23:41:31 d2.engine.defaults]: Auto-scaling the config to batch_size=2, learning_rate=0.0005, max_iter=700, warmup=2000.
[07/09 23:41:37 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=

model_final_68b088.pkl: 421MB [00:03, 116MB/s]                           


[07/09 23:42:11 d2.engine.train_loop]: Starting training from iteration 0


/opt/conda/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


[07/09 23:43:00 d2.utils.events]:  eta: 0:26:31  iter: 19  total_loss: 1.079  loss_cls: 0.7121  loss_box_reg: 0.2953  loss_rpn_cls: 0.0369  loss_rpn_loc: 0.009739  time: 2.4548  data_time: 0.0257  lr: 1.4058e-05  max_mem: 6049M
[07/09 23:43:51 d2.utils.events]:  eta: 0:27:11  iter: 39  total_loss: 0.8502  loss_cls: 0.5385  loss_box_reg: 0.2811  loss_rpn_cls: 0.04396  loss_rpn_loc: 0.009119  time: 2.5144  data_time: 0.0060  lr: 2.8329e-05  max_mem: 6049M
[07/09 23:44:45 d2.utils.events]:  eta: 0:26:42  iter: 59  total_loss: 0.8527  loss_cls: 0.399  loss_box_reg: 0.4593  loss_rpn_cls: 0.02766  loss_rpn_loc: 0.006943  time: 2.5717  data_time: 0.0072  lr: 4.2601e-05  max_mem: 6049M
[07/09 23:45:36 d2.utils.events]:  eta: 0:25:57  iter: 79  total_loss: 0.66  loss_cls: 0.2913  loss_box_reg: 0.3013  loss_rpn_cls: 0.02032  loss_rpn_loc: 0.00696  time: 2.5692  data_time: 0.0067  lr: 5.6872e-05  max_mem: 6049M
[07/09 23:46:33 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inferen

## Loading the model

In [15]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.35   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

## Loading the test dataset and registring it.

In [16]:
test = pd.read_csv('../input/detectron2-sea-turtle-data-loader/SampleSubmission.csv')
d = 'test'
DatasetCatalog.register("turtle_" + d, lambda d=d: get_turtle_box(image_folder,test))

In [17]:
testset_dicts = get_turtle_box(image_folder,test)

## Inference and results.

In [18]:
data_result = []
for d in testset_dicts:
    print(d["file_name"])
    print(d["image_id"])
    im = cv2.imread(d["file_name"])
    height, width = im.shape[:2]
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    print(outputs['instances'].pred_boxes)
    result = outputs['instances'].pred_boxes.tensor.to("cpu").numpy()
    
    if(result.shape[1]!= 4):
        assert 1==2
    if result.shape[0] > 1:
        
        result = result[0]
    result = result.flatten()
    
    if len(result) != 4:
        print(d["file_name"])
        print(result)
    final_result = []
    final_result.append(d["image_id"])
    
    for i in range(len(result)):
        if i < 2:  
            if i%2 == 0:
                final_result.append(result[i]/width)
            else:
                final_result.append(result[i]/height)
                
        else:
            if i%2 == 0:
                final_result.append((result[i]-result[i-2])/width)
            else:
                final_result.append((result[i]-result[i-2])/height)
            
    data_result.append(final_result)

IMAGES_512/E02ABB6C.JPG
E02ABB6C
Boxes(tensor([[173.1180, 203.9915, 383.3274, 365.9210]], device='cuda:0'))
IMAGES_512/9CA35CC5.JPG
9CA35CC5
Boxes(tensor([[221.0568, 238.7551, 353.0796, 340.1317]], device='cuda:0'))
IMAGES_512/F5B0AB72.JPG
F5B0AB72
Boxes(tensor([[226.9574, 140.5882, 305.1251, 256.4944]], device='cuda:0'))
IMAGES_512/58AE1466.JPG
58AE1466
Boxes(tensor([[203.5870, 182.9815, 323.8069, 360.2692]], device='cuda:0'))
IMAGES_512/B4982B53.JPG
B4982B53
Boxes(tensor([[ 93.5213,  46.0124, 467.1386, 383.1892]], device='cuda:0'))
IMAGES_512/387683D3.JPG
387683D3
Boxes(tensor([[141.1955, 132.4845, 306.7370, 241.7528],
        [258.5742,  31.6292, 469.5852, 201.5803]], device='cuda:0'))
IMAGES_512/8F46E061.JPG
8F46E061
Boxes(tensor([[227.0079, 152.8911, 376.0995, 261.4165]], device='cuda:0'))
IMAGES_512/CF944985.JPG
CF944985
Boxes(tensor([[287.7787, 111.1601, 452.0949, 301.2548]], device='cuda:0'))
IMAGES_512/72A118D0.JPG
72A118D0
Boxes(tensor([[296.5618, 106.2188, 411.7884, 242.0674

## To csv for submission.

In [19]:
result_df = pd.DataFrame(data_result,columns=train.columns)

In [20]:
print(data_result[:8])

[['E02ABB6C', 0.3381209969520569, 0.5312279065450033, 0.41056525707244873, 0.4216914176940918], ['9CA35CC5', 0.43175163865089417, 0.6217581828435262, 0.25785699486732483, 0.2640013297398885], ['F5B0AB72', 0.44327613711357117, 0.48815351062350804, 0.15267124772071838, 0.40245209799872506], ['58AE1466', 0.397630900144577, 0.47651441891988117, 0.23480448126792908, 0.4616866906483968], ['B4982B53', 0.18265877664089203, 0.11982391277949016, 0.7297213077545166, 0.8780646324157715], ['387683D3', 0.27577248215675354, 0.3450115919113159, 0.32332322001457214, 0.28455297152201336], ['8F46E061', 0.44337478280067444, 0.39815394083658856, 0.29119452834129333, 0.28261828422546387], ['CF944985', 0.5620678663253784, 0.28947951396306354, 0.32092994451522827, 0.4950382709503174]]


In [21]:
for i in range(len(data_result)):
    if len(data_result[i]) != 5:
        print(i)

In [22]:
result_df = result_df.fillna(0)

In [23]:

result_df.to_csv('result_v26.csv', index=False)

In [24]:
for dirname, _, filenames in os.walk('/kaggle/output'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Achieved a score of 0.9135 on the public leaderboard.